In [117]:
import gurobipy as gp
from gurobipy import GRB
import math
import numpy as np
import pandas as pd

# Problem 1

### 1. What are APLO's aims in undertaking this project? For this project to be a showcase for the company, what would you recommend (in non-math terms) that they should try to optimize?

APLO's primary aim is to establish a strong market presence and reputation in Indonesia's growing economy by providing modern, ecological, and reliable LED street lighting to win more contracts throughout Indonesia’s 34 provinces.


They should try to optimize:
- **Component Quality and Reliability:** Choose components with the lowest defect rates to ensure longevity and reduce the likelihood of failures during the five-year guarantee period. This will reduce maintenance costs and build trust in APLO's commitment to quality.
- **Supplier Compatibility:** Be careful of the compatibility of components from different suppliers to avoid any operational issues post-installation. Components that are known to be incompatible should not be selected together.
- **Cost-Effectiveness:** Manage costs carefully to stay within the budget cap per street light. This involves balancing initial costs with potential future savings. For example, a slightly more expensive component that promises lower defect rates might be more cost-effective in the long run.

### 2. How should we evaluate the total expected cost of a set of components over the five-year guarantee period? Explain any assumptions or approximations that you have made in doing so.

<span style="font-size:1.5em;"> Expercted Costs </span>
- **Initial Cost:** The purchase cost of each component. <br>
- **Defect Rate:** The probability that the component will fail within the guarantee period. <br>
- **Replacement Cost:** The cost of replacing a component if it fails during the five-year guarantee period. <br>
- **UL Certification:** Whether the component is UL Certified, as this might affect the likelihood of failure and eligibility for the project. <br>
- **Extra Pre-Assembly Cost:** Additional costs associated with pre-assembly of certain components. <br>

<span style="font-size:1.5em;"> Assumptions/Approximations </span>
- As it is stated in the case, the failure of one component does not affect the others <br>
- The replacement cost is constant over the five-year period (not accounting for inflation, changes in supplier pricing, etc) <br>
- The only cost incurred from a component failure is the replacement of the component itself.  <br>
- The defect rate remains uniform over the five-year period <br>
- THe costs provided from the case are accurate and will not change over time <br>

### 3. We will examine two possible ways of approaching this problem. First, write down the model that minimizes total expected cost over the five-year guarantee period, assuming that each component fails at most once over five years. (For incompatibility issues and complementary features, use only those mentioned in the case.)

Let $ x_{ij} $ be a binary decision variable that equals 1 if component $ i $ from supplier $ j $ is chosen, and 0 otherwise.

Objective Function:
- Minimize $ Z = \sum_{i=1}^{n}\sum_{j=1}^{m_i} (x_{ij} \cdot (\text{Cost}_{ij} + (\text{Defect Rate}_{ij} \cdot \text{Cost}_{ij}) + \text{Extra Pre-Assembly Cost}_{ij})) $

Where:
- $ n $ is the total number of components.
- $ m_i $ is the number of suppliers for component $ i $.
- $ \text{Cost}_{ij} $ is the initial cost of component $ i $ from supplier $ j $.
- $ \text{Defect Rate}_{ij} $ is the defect rate of component $ i $ from supplier $ j $.
- $ \text{Extra Pre-Assembly Cost}_{ij} $ is the extra pre-assembly cost for component $ i $ from supplier $ j $ (this could be zero if no pre-assembly is required).

Constraints:
1. Component Selection Constraint: For each component $ i $, exactly one supplier must be chosen. 

   $ \sum_{j=1}^{m_i} x_{ij} = 1 $ for all $ i $

2. UL Certification Constraint: At least seven of the ten components must be UL certified. 

   $ \sum_{i=1}^{n}\sum_{j=1}^{m_i} (x_{ij} \cdot \text{UL Certified}_{ij}) \geq 7 $

3. Incompatibility Constraint: If components from suppliers Sup13 and Sup23 are incompatible, they cannot be selected together. 

   $ x_{1\text{Sup13}} + x_{2\text{Sup23}} \leq 1 $.

4. Complementary Features Constraint: If components from suppliers Sup21 and Sup42 are complementary, they must be selected together. 

   $ x_{2\text{Sup21}} = x_{4\text{Sup42}} $

5. Budget Constraint: The total cost must not exceed the budget cap of $5,400 per street light. 

   $ \sum_{i=1}^{n}\sum_{j=1}^{m_i} (x_{ij} \cdot (\text{Cost}_{ij} + (\text{Defect Rate}_{ij} \cdot \text{Cost}_{ij}) + \text{Extra Pre-Assembly Cost}_{ij})) \leq \text{Budget Cap} $

6. Binary Decision Variables: Ensure $ x_{ij} $ can only take values 0 or 1. 

   $ x_{ij} \in \{0,1\} $ for all $ i, j $.

### 4. Implement and solve this model in Jupyter Notebook using Gurobi. What is the cost of Solution 1? What is the probability that no component in Solution 1 fails over the five-year guarantee period? What is the probability that at most one of the components in Solution 1 fails over the five-year guarantee period?

In [118]:
m = gp.Model()

## 1) Define variables: 

### Whether we use a given supplier for a component

### Or for a more compact way of defining your variables, you can use this code: 
    

In [119]:
# Let's just input how many suppliers we have for each component
J = [4,5,4,5,4,4,4,5,5,3]

# Create list of indices we need for the variables 
indices=[]
for i in range(10):
    for j in range(J[i]): 
        indices=indices+[(i,j)]
        
Sup=m.addVars(indices, vtype=GRB.BINARY, name='Sup')
# Now you can call each variable by its corrdinates: example Sup[0,2]

### We need to input some data. The easiest way to do this is to store it in an array and call the data by index when we need it.

In [120]:
# read the data 
ALPO= pd.read_csv("APLO Raw Data.csv")

In [121]:
ALPO.head()

,Component #,Supplier,Defect Rate,UL Certified,Cost (NT$),Extra Pre-Assembly Cost (NT$)
0,#1,Sup11,0.037,1,1380.0,0
1,#1,Sup12,0.017,1,1668.0,0
2,#1,Sup13,0.015,1,1464.0,0
3,#1,Sup14,0.018,1,1620.0,0
4,#2,Sup21,0.043,1,278.4,0


In [122]:
UL = [[0 for x in range(J[y])] for y in range(10)] 
Cost = [[0 for x in range(J[y])] for y in range(10)]
Defect = [[0 for x in range(J[y])] for y in range(10)]
ExtraCost = [[0 for x in range(J[y])] for y in range(10)] 

indices = [0] + [sum(J[:i+1]) for i in range(len(J))]

for i in range(10):
    UL[i][0:J[i]] = ALPO['UL Certified'][indices[i]:indices[i+1]].tolist()
    Cost[i][0:J[i]] = ALPO['Cost (NT$)'][indices[i]:indices[i+1]].tolist()
    Defect[i][0:J[i]] = ALPO['Defect Rate'][indices[i]:indices[i+1]].tolist()
    ExtraCost[i][0:J[i]] = ALPO['Extra Pre-Assembly Cost (NT$)'][indices[i]:indices[i+1]].tolist()

In [123]:
Cost[1]

[278.4, 289.2, 252.0, 264.0, 270.0]

### Now let's fill in the model! 

In [150]:
#Total budget
Budget_Cap = 5400

# Define objective function
m.setObjective(gp.quicksum(Sup[i, j] * (Cost[i][j] + Cost[i][j] * Defect[i][j] + ExtraCost[i][j]) for i in range(10) for j in range(J[i])),GRB.MINIMIZE)

# Incompatibility issues: Sup13 and Sup23 are incompatible
m.addConstr(Sup[0, 2] + Sup[1, 2] <= 1)

# Complementary features: Sup21 and Sup42 are complementary
m.addConstr(Sup[1, 0] - Sup[3, 1] == 0)

# UL Certification Constraint: At least seven of the ten components must be UL certified
m.addConstr(gp.quicksum(Sup[i, j] * UL[i][j] for i in range(10) for j in range(J[i])) >= 7)

# Exactly one supplier must be selected for each component
for i in range(10):
    m.addConstr(gp.quicksum(Sup[i, j] for j in range(J[i])) == 1)

# Budget constraint
m.addConstr(sum(Sup[i,j]*(Cost[i][j] + Defect[i][j] * Cost[i][j] + ExtraCost[i][j])for i in range(10) for j in range(J[i])) <= Budget_Cap)


<gurobi.Constr *Awaiting Model Update*>

### Solve the model and get the objective value and any other calculations you need! 

In [151]:
m.Params.LogToConsole = 0
status = m.optimize()
ObjectiveValue=m.objVal

# Output and analysis
print(f"Total Expected Cost is: ${ObjectiveValue}")

prob_no_fail = 1
for i in range(10):
    for j in range(J[i]):
        if Sup[i, j].X > 0.5: 
            prob_no_fail *= (1 - Defect[i][j])
print("Probability that no component fails: {:.2%}".format(prob_no_fail))

prob_one_fail = 0
for i in range(10):
    for j in range(J[i]):
        if Sup[i, j].X > 0.5:  
            prob_fail_i = Defect[i][j] 
            adjusted_prob_no_fail = prob_no_fail / (1 - Defect[i][j])
            prob_one_fail += prob_fail_i * adjusted_prob_no_fail

prob_at_most_one_fail = prob_no_fail + prob_one_fail

print("Probability that at most one component fails: {:.2%}".format(prob_at_most_one_fail))


Total Expected Cost is: $5057.478
Probability that no component fails: 56.81%
Probability that at most one component fails: 90.04%


### 5. Write down the non-linear model that maximizes the probability that no components fail over the five- year guarantee period, subject to budget constraints. Why will this be more diffcult to implement?

Objective Function:

- Maximize $ Z = \prod_{i=1}^{n} \prod_{j=1}^{m_i} (1-p_{ij})^{x_{ij}} $

where:
- $p_{ij}$ is the probability of component $i$ from supplier $j$ not failing,
- $x_{ij}$ is a binary decision variable that equals 1 if component $i$ from supplier $j$ is chosen, and 0 otherwise.

Constraints remain the same as before:
1. Component Selection Constraint: For each component $ i $, exactly one supplier must be chosen. 
2. UL Certification Constraint: At least seven of the ten components must be UL certified. 
3. Incompatibility Constraint: If components from suppliers Sup13 and Sup23 are incompatible, they cannot be selected together. 
4. Complementary Features Constraint: If components from suppliers Sup21 and Sup42 are complementary, they must be selected together. 
5. Budget Constraint: The total cost must not exceed the budget cap of $5,400 per street light. 
6. Binary Decision Variables: Ensure $ x_{ij} $ can only take values 0 or 1. 

<span style="font-size:1.5em;"> Why will this be more diffcult to implement: </span>
- Gurobi does not have built in non-linear functions
- Many non-linear problems are non-convex. And Non-convex problems can have multiple local optima – points that are better than all other feasible points in their vicinity but are not the best overall solution. Identifying the global optimum is significantly more challenging than in convex problems.

### 6. To implement this, we approximate the objective function by the sum of probability of failure for all selected components. Implement and solve this model in Jupyter Notebook. We call the resulting output Solution 2. What is the cost of Solution 2?

In [153]:
m2 = gp.Model()

In [154]:
J = [4,5,4,5,4,4,4,5,5,3]

indices=[]
for i in range(10):
    for j in range(J[i]): 
        indices=indices+[(i,j)]

Sup2=m2.addVars(indices, vtype=GRB.BINARY, name='Sup')

In [156]:
# Define objective function
m2.setObjective(gp.quicksum(Sup2[i,j]*Defect[i][j] for i in range(10) for j in range(J[i])), GRB.MINIMIZE)

# Incompatibility issues: Sup13 and Sup23 are incompatible
m2.addConstr(Sup2[0, 2] + Sup2[1, 2] <= 1)

# Complementary features: Sup21 and Sup42 are complementary
m2.addConstr(Sup2[1, 0] - Sup2[3, 1] == 0)

# UL Certification Constraint: At least seven of the ten components must be UL certified
m2.addConstr(gp.quicksum(Sup2[i, j] * UL[i][j] for i in range(10) for j in range(J[i])) >= 7)

# Exactly one supplier must be selected for each component
for i in range(10):
    m2.addConstr(gp.quicksum(Sup2[i, j] for j in range(J[i])) == 1)

# Budget constraint
m2.addConstr(gp.quicksum(Sup2[i,j]*(Cost[i][j] + Defect[i][j] * Cost[i][j] + ExtraCost[i][j])for i in range(10) for j in range(J[i])) <= Budget_Cap)

<gurobi.Constr *Awaiting Model Update*>

In [159]:
m2.Params.LogToConsole = 0
m2.optimize()
ObjectiveValue2=m2.objVal

total_cost = sum(Sup2[i, j].X * (Cost[i][j] + Defect[i][j] * Cost[i][j] + ExtraCost[i][j]) for i in range(10) for j in range(J[i]))
print(f"Total Expected Cost of Solution 2: ${total_cost}")
print(f"The optimal value of Solution 2 objective function: {ObjectiveValue2} \n")

if m2.status == GRB.OPTIMAL:
    print("Optimal Decisions of Solution 2:")
    for i in range(10):
        for j in range(J[i]):
            if Sup2[i, j].X > 0.5: 
                print(f"Component {i+1} from Supplier {j+1} has been selected.")
print()

prob_no_fail_components = [1 - Defect[i][j-1] for i, j in enumerate([3, 2, 3, 3, 4, 4, 1, 5, 1, 2])]

prob_no_fail = 1
for prob in prob_no_fail_components:
    prob_no_fail *= prob

print(f"Probability that no component fails of Solution 2: {prob_no_fail:.2%}")

prob_one_fail = prob_no_fail  
for i, prob in enumerate(prob_no_fail_components):
    prob_no_fail_except_current = prob_no_fail / prob
    prob_one_fail += (1 - prob) * prob_no_fail_except_current

print(f"Probability that at most one component fails of Solution 2: {prob_one_fail:.2%}")


Total Expected Cost of Solution 2: $5382.823599999999
The optimal value of Solution 2 objective function: 0.22699999999999998 

Optimal Decisions of Solution 2:
Component 1 from Supplier 3 has been selected.
Component 2 from Supplier 2 has been selected.
Component 3 from Supplier 3 has been selected.
Component 4 from Supplier 3 has been selected.
Component 5 from Supplier 4 has been selected.
Component 6 from Supplier 4 has been selected.
Component 7 from Supplier 1 has been selected.
Component 8 from Supplier 5 has been selected.
Component 9 from Supplier 1 has been selected.
Component 10 from Supplier 2 has been selected.

Probability that no component fails of Solution 2: 79.38%
Probability that at most one component fails of Solution 2: 98.03%


### 7. Which of the two models would you recommend that APLO use, and why?

Given the significantly increased probabilities of no component failure and at most one component failing, along with the relatively modest increase in cost, **Model 2** would be recommended for APLO.

<span style="font-size:1.5em;"> Reasons: </span>

- Model 1 is more cost-effective but offers lower reliability in terms of both the probability of no component failure and the probability that at most one component fails.
- **Model 2** while more expensive, significantly improves the reliability of the system.

Moreover, higher reliability might lead to lower maintenance costs in the long term, potentially offsetting the higher initial cost of Model 2. Also, customer satisfaction and brand reputation could also be significantly impacted by the reliability of the products from **Model 2**.


---

# Problem 2

### 1. Formulate this problem using a linear discrete optimization model.

Decision Variables:
Let $ x_j $ be a binary variable that equals 1 if project $ j $ is selected, and 0 otherwise.

Objective Function:

$ \text{Maximize} \; Z = \sum_{j=1}^{10} P_j \cdot x_j $

where:

- $ P_j $ is the estimated long-run profit for project $ j $

Constraints:
1. **Capital Constraint**: The total initial capital required for the selected projects cannot exceed $175 million.
   
   $ \sum_{j=1}^{10} C_j \cdot x_j \leq 175 $

   where $ C_j $ is the initial capital required for project $ j $.

2. **Project Selection Constraint**: At least 4 projects should be chosen. 

   $ \sum_{j=1}^{10} x_j \geq 4 $

3. **Non-negativity/Binary Restriction**: The decision variables are binary, which is inherently non-negative.

   $ x_j \in \{0, 1\} \; \text{for} \; j = 1, ..., 10 $


### 2. In addition, the following constraints have to be met. Please formulate each one independently 

(a) If they invest in 3 or more projects with an even subscript (i.e. 2 f2; 4; 6; 8; 10g), they need to invest in at least one project with an odd subscript (i.e. 2 f1; 3; 5; 7; 9g).

$ \sum_{j \in \{2,4,6,8,10\}} x_j \geq 3 \cdot y $

$ \sum_{j \in \{1,3,5,7,9\}} x_j \geq y $

Where:
- $ y \in \{0,1\} $

(b) If they choose to invest in all of the projects, it is estimated that they will lose profit on the long run of $50 Millions.

New objective function will be:

$ \text{Maximize} \; Z = \sum_{j=1}^{10} P_j \cdot x_j - 50 \cdot y $

With the constraint that:

$ \sum_{j=1}^{10} x_j \geq 10y $

Where:
- $ y \in \{0,1\} $

(c) If the total initial capital required for the projects exceeds $150 Millions, then the expected long-run profit needs to be at least $300 Millions.

$ \sum_{j=1}^{10} C_j \cdot x_j \leq 150 + My $

$ \sum_{j=1}^{10} P_j \cdot x_j \geq 300y $

Where:
- $ y \in \{0,1\} $

(d) The total initial capital required for the projects takes discrete values: it needs to be either $100, $125, $150 or $175 millions.

Let $ C_{100}, C_{125}, C_{150}, C_{175} $ be binary variables for each capital level.

$ \sum_{j=1}^{10} C_j \cdot x_j = 100 \cdot C_{100} + 125 \cdot C_{125} + 150 \cdot C_{150} + 175 \cdot C_{175} $

$ C_{100} + C_{125} + C_{150} + C_{175} = 1 $

### 3. Assume that you are instead minimizing the initial capital required across chosen projects, with some generic constraints. After you solve the model using an optimization solver, your code shows an optimal solution that suggests to undertake projects 1, 2, 3, 4, and 6. If there is another (different) optimal solution that yields the same objective function value, what constraint would you add to the problem in order to find this alternate solution?

I would add additional constraint -

$ x_1 + x_2 + x_3 + x_4 + x_5 < 5 $

This make sure the solver gives different solution